In [1]:
import os
from openai import OpenAI

API_KEY = os.getenv("OPENAI_API_KEY") 
GPT_MODEL_NAME="gpt-4o-mini"

client = OpenAI(api_key=API_KEY)

In [10]:
INTRO_PROMPT = (
    "너는 감정 케어를 도와주는 대화형 AI 'Dolbom'야.\n"
    "상대방의 감정에 공감하고, 따뜻하고 다정한 말투로 위로와 응원을 건네는 역할을 해.\n\n"
    "대화를 시작하기 전, 다음의 설정 정보를 참고해야 해:\n"
    "- [말투]: 문장 끝 어미에 반영되는 말투 스타일이야. 예: 존댓말, 반말 등\n"
    "- [성격]: 응답의 어조와 분위기에 영향을 주는 Dolbom의 성격이야. 예: 내향적, 외향적\n"
    "- [참고 설정]: 사용자가 전달한 추가 정보로, 대화 중 필요할 수 있는 중요한 참고사항이야\n"
    "- [훈련 명령 목록]: 사용자가 Dolbom에게 훈련시킨 명령어와 대응 행동이야. 대화 중 관련된 문맥이 나타나면 활용해\n\n"
    "반드시 한국어로만 응답하고, 설정된 말투와 성격을 반영해서 대화해.\n"
    "너무 길거나 장황하지 않게, 핵심적이고 감성적으로 응답하는 것이 좋아."
)

def build_input_messages(user_message: str, character: dict, trainings: list,
                         chat_history: list = None, max_trainings: int = 5) -> list:
    messages = [
        {"role": "system", "content": INTRO_PROMPT},
        {"role": "system", "content": f"[말투] → {character.get('speech', '존댓말')}"},
        {"role": "system", "content": f"[성격] → {character.get('character', '내향적')}"},
        {"role": "system", "content": f"[참고 설정] → {character.get('resSetting', '')}"}
    ]

    if trainings:
        limited = trainings[:max_trainings]
        training_lines = "\n".join(
            f"- {t.get('trainingText', '')} → {t.get('recognizedGesture', '')}"
            for t in limited
        )
        messages.append({"role": "system", "content": f"[훈련 명령 목록]\n{training_lines}"})

    if chat_history:
        messages.extend(chat_history)

    # 마지막 사용자 입력 추가
    messages.append({"role": "user", "content": user_message})

    return messages

In [11]:
def moderate_text(text: str) -> bool:
    """
    주어진 텍스트가 OpenAI Moderation에 의해 차단되는지 확인.
    부적절하면 True(차단 필요 -> 응답 생성 X)
    적절하면 False(정상 처리 -> 응답 생성 진행)
    """
    moderation = client.moderations.create(input=text)
    flagged = any(result.flagged for result in moderation.results)
    return flagged

In [14]:
def generate_reply(user_message: str,
                   character: dict,
                   trainings: list,
                   chat_history: list = None,
                   stream: bool = True,
                   model: str = "gpt-4o-mini"):
    """
    사용자 입력, 캐릭터 설정, 훈련 목록, 히스토리를 바탕으로 응답 생성 (stream/non-stream 지원)

    Returns:
        - response: stream generator or 단일 응답
        - response_id: 응답 ID (non-stream 시 반환됨)
    """
    if moderate_text(user_message):
        return "⚠️ 입력 내용에 부적절한 요소가 포함되어 있어요. ⚠️", None

    input_messages = build_input_messages(
        user_message=user_message,
        character=character,
        trainings=trainings,
        chat_history=chat_history
    )

    tools = [{"type": "web_search_preview"}] if "웹 검색" in user_message else None

    response = client.responses.create(
        model=model,
        input=input_messages,
        tools=tools,
        # max_tokens=2048,  # Response API에서는 max_tokens을 직접 설정 불가능. 모델의 context window에 맞춰 자동 조절됨.
        temperature=0.7,
        top_p=0.9,
        stream=stream
    )

    # stream일 경우: generator 반환, ID 없음
    if stream:
        return response, None
    else:
        return response.output_text.strip(), response.id

In [19]:
if __name__ == "__main__":
    # 예시 성격 설정
    test_character = {
        "speech": "존댓말",
        "character": "내향적",
        "resSetting": "땅콩 알레르기 있음"
    }

    # 예시 훈련 명령어
    test_trainings = [
        {
            "trainingText": "도와줘",
            "recognizedGesture": "손 흔들기"
        },
        {
            "trainingText": "날씨",
            "recognizedGesture": "오늘의 날씨 데이터 가져오기"
        }
    ]

    # 예시 히스토리 (필요 없으면 빈 리스트로)
    test_history = [
        {"role": "user", "content": "기분이 이상해..."},
        {"role": "assistant", "content": "걱정 마세요. 제가 도와드릴게요."}
    ]

    # 사용자 입력
    user_input = "고마워. 지금은 조금 나아졌어."

    # stream 모드 선택
    use_stream = True

    # 응답 생성
    response, response_id = generate_reply(
        user_message=user_input,
        character=test_character,
        trainings=test_trainings,
        chat_history=test_history,
        stream=use_stream
    )

    # 출력 처리
    if isinstance(response, str):
        print(response)  # moderation 차단 메시지
    elif use_stream:
        print("\n[Dolbom 응답 시작 - 실시간 stream]\n")
        for chunk in response:
            # delta 이벤트일 경우만 출력
            if chunk.__class__.__name__ == "ResponseTextDeltaEvent":
                print(chunk.delta, end="", flush=True)
        print("\n\n[Dolbom 응답 종료]")
    else:
        print("\n[Dolbom 응답 - 전체 텍스트]\n")
        print(response)
        print(f"\n[응답 ID] → {response_id}")


[Dolbom 응답 시작 - 실시간 stream]

다행입니다. 이렇게 이야기 나누는 것이 조금이라도 도움이 된다면 좋겠어요. 더 이야기하고 싶은 게 있으면 언제든지 말씀해 주세요.

[Dolbom 응답 종료]
